In [1]:
# !pip3 install sqlalchemy
# !pip install psycopg2-binary

In [14]:
import psycopg 
from sqlalchemy import create_engine 
import geopandas as gpd

In [16]:
engine = create_engine('postgresql://localhost/postgres')

con = psycopg.connect( host = 'localhost', user = 'postgres', dbname = 'postgres', port='5432', password='',
        keepalives=1, keepalives_idle= 60, keepalives_interval= 15, keepalives_count=5, autocommit= True)

In [4]:
gdf = gpd.read_file('doctors.geojson',  driver='GeoJson')
gdf.head()

,index,geohash,num,place_id,vicinity,name,business_status,lat,lng,geometry
0,0,sne00,3,ChIJq6qmSo6yjRIRo0Zgqy-1Ff0,Beni Yenni,Dr. B. Zaddoud,OPERATIONAL,36.568672,4.220461,POINT (4.22046 36.56867)
1,0,sne00,3,ChIJF5nEo8itjRIRk67t1f685S8,Boulevard Colonel Amirouche,Dr Ait ali,OPERATIONAL,36.573897,4.306021,POINT (4.30602 36.57390)
2,1,sndbw,3,ChIJw918RiyxjRIRnBmBIgMOadk,Irdjen,Docteur Osmani.M Chirurgien Dentiste,OPERATIONAL,36.659399,4.154959,POINT (4.15496 36.65940)
3,1,sndbw,3,ChIJFYa4csWxjRIRdRoefoHSOdY,Tizi Rached,Cabinet Dr BenMessaoud de Pédiatrie عيادة طب ا...,OPERATIONAL,36.697355,4.193175,POINT (4.19317 36.69736)
4,1,sndbw,3,ChIJPx7BCe-xjRIRugS7SHnTUNM,Tizi Rached,Dr Mekharef épouse Zaouia,OPERATIONAL,36.697325,4.193222,POINT (4.19322 36.69732)


In [7]:
gdf.to_postgis(con=engine, name='doctors', if_exists='replace')

In [8]:
s= 'select * from doctors ST_MakeEnvelope(left, bottom, right, top, 4326)'
fetch_json(s)

In [18]:
def fetch_json (sql:str):
    cur = con.cursor()
    sql = f"""
        SELECT json_build_object(
            'type', 'FeatureCollection',
            'features', json_agg(ST_AsGeoJSON(t.*)::json)
        )
        FROM (
       {sql}
            ) t;
        """
    cur.execute (sql) 
    
    return cur.fetchone()[0]

In [19]:
fetch_json(s)

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'geometry': {'type': 'Point', 'coordinates': [4.2204609, 36.5686722]},
   'properties': {'index': 0,
    'geohash': 'sne00',
    'num': 3,
    'place_id': 'ChIJq6qmSo6yjRIRo0Zgqy-1Ff0',
    'vicinity': ' Beni Yenni',
    'name': 'Dr. B. Zaddoud',
    'business_status': 'OPERATIONAL',
    'lat': 36.5686722,
    'lng': 4.2204609}},
  {'type': 'Feature',
   'geometry': {'type': 'Point', 'coordinates': [4.3060213, 36.5738968]},
   'properties': {'index': 0,
    'geohash': 'sne00',
    'num': 3,
    'place_id': 'ChIJF5nEo8itjRIRk67t1f685S8',
    'vicinity': 'Boulevard Colonel Amirouche',
    'name': 'Dr Ait ali',
    'business_status': 'OPERATIONAL',
    'lat': 36.5738968,
    'lng': 4.3060213}},
  {'type': 'Feature',
   'geometry': {'type': 'Point', 'coordinates': [4.1549588, 36.6593991]},
   'properties': {'index': 1,
    'geohash': 'sndbw',
    'num': 3,
    'place_id': 'ChIJw918RiyxjRIRnBmBIgMOadk',
    'vicinity': ' Ird